# Enhancing Singapore Airlines' Service Through Automated Sentiment Analysis of Customer Reviews



**Motivation**



## Singapore Airlines Customer Reviews Dataset Information

The [Singapore Airlines Customer Reviews Dataset](https://www.kaggle.com/datasets/kanchana1990/singapore-airlines-reviews) aggregates 10,000 anonymized customer reviews, providing a broad perspective on the passenger experience with Singapore Airlines. 

The dimensions are shown below:
- **`published_date`**: Date and time of review publication.
- **`published_platform`**: Platform where the review was posted.
- **`rating`**: Customer satisfaction rating, from 1 (lowest) to 5 (highest).
- **`type`**: Specifies the content as a review.
- **`text`**: Detailed customer feedback.
- **`title`**: Summary of the review.
- **`helpful_votes`**: Number of users finding the review helpful.

## Importing Libraries

Please uncomment the code box below to pip install relevant dependencies for this notebook.

In [2]:
!pip3 install -r ../requirements.txt

In [1]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

# Plotting and Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Language Detection packages
# `langdetect` for detecting language
from langdetect import detect as langdetect_detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
# `langid` for an alternative language detection method
from langid import classify as langid_classify

# Text Preprocessing and NLP
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords

# Tokenizing sentences/words
from nltk.corpus import wordnet

# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer
import nltk
# Regular expressions for text pattern matching
import re

# Word Cloud generation
from wordcloud import WordCloud

# For generating n-grams
from nltk.util import ngrams
from collections import Counter

# Libraries for Word2Vec and Logistic Regression
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, make_scorer

In [2]:
data = pd.read_csv("final_df.csv")

# Word2Vec + ComplementNB

In [6]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Tokenize the processed reviews for Word2Vec training
tokenized_reviews = [review.split() for review in data['processed_full_review']]

# Train the Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, sg=1, workers=4, seed=42)

# Function to compute the average word vectors for each review
def get_average_word2vec(review, model, vector_size):
    words = review.split()
    word_vecs = [model.wv[word] for word in words if word in model.wv]
    if word_vecs:
        return np.mean(word_vecs, axis=0)
    else:
        return np.zeros(vector_size)

# Create the feature matrix by averaging word vectors for each review
vector_size = w2v_model.vector_size
X = np.array([get_average_word2vec(review, w2v_model, vector_size) for review in data['processed_full_review']])

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2, random_state=42)

# Initialize and train the Complement Naive Bayes model
nb_model = ComplementNB(alpha=5.0)
nb_model.fit(X_train, y_train)

# Make predictions
nb_predictions = nb_model.predict(X_test)

# Evaluate the model
print("Complement NB Accuracy:", accuracy_score(y_test, nb_predictions))
print("Complement NB Classification Report:\n", classification_report(y_test, nb_predictions, digits=4))

ValueError: Negative values in data passed to ComplementNB (input X)

# Word2Vec + RF

In [5]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Tokenize the processed reviews for Word2Vec training
tokenized_reviews = [review.split() for review in data['processed_full_review']]

# Train the Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, sg=1, workers=4, seed=42)

# Function to compute the average word vectors for each review
def get_average_word2vec(review, model, vector_size):
    words = review.split()
    word_vecs = [model.wv[word] for word in words if word in model.wv]
    if word_vecs:
        return np.mean(word_vecs, axis=0)
    else:
        return np.zeros(vector_size)

# Create the feature matrix by averaging word vectors for each review
vector_size = w2v_model.vector_size
X = np.array([get_average_word2vec(review, w2v_model, vector_size) for review in data['processed_full_review']])

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
rf_predictions = rf_model.predict(X_test)

# Evaluate the model
print("Random Forest Accuracy:", accuracy_score(y_test, rf_predictions))
print("Random Forest Classification Report:\n", classification_report(y_test, rf_predictions, digits=4))


Random Forest Accuracy: 0.8428819444444444
Random Forest Classification Report:
               precision    recall  f1-score   support

    Negative     0.7510    0.7766    0.7636       470
     Neutral     0.5075    0.1491    0.2305       228
    Positive     0.8812    0.9608    0.9193      1606

    accuracy                         0.8429      2304
   macro avg     0.7132    0.6288    0.6378      2304
weighted avg     0.8177    0.8429    0.8194      2304



# Word2Vec + log regression

In [ ]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Tokenize the processed reviews for Word2Vec training
tokenized_reviews = [review.split() for review in data['processed_full_review']]

# Train the Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, sg=1, workers=4, seed=42)

# Function to compute the average word vectors for each review
def get_average_word2vec(review, model, vector_size):
    words = review.split()
    word_vecs = [model.wv[word] for word in words if word in model.wv]
    if word_vecs:
        return np.mean(word_vecs, axis=0)
    else:
        return np.zeros(vector_size)

# Create the feature matrix by averaging word vectors for each review
vector_size = w2v_model.vector_size
X = np.array([get_average_word2vec(review, w2v_model, vector_size) for review in data['processed_full_review']])

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2, random_state=42)

# Initialize and train the Logistic Regression model
clf = LogisticRegression(random_state=42, multi_class='multinomial', solver='lbfgs', max_iter=100)
clf.fit(X_train, y_train)

# Make predictions
clf_predictions = clf.predict(X_test)

# Evaluate the model
print("Logistic Regression Accuracy:", accuracy_score(y_test, clf_predictions))
print("Logistic Regression Classification Report:\n", classification_report(y_test, clf_predictions, digits=4))

Logistic Regression Accuracy: 0.8511284722222222
Logistic Regression Classification Report:
               precision    recall  f1-score   support

    Negative     0.7759    0.7957    0.7857       470
     Neutral     0.4248    0.2105    0.2815       228
    Positive     0.9005    0.9583    0.9285      1606

    accuracy                         0.8511      2304
   macro avg     0.7004    0.6549    0.6652      2304
weighted avg     0.8280    0.8511    0.8354      2304



c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
